<a href="https://colab.research.google.com/github/Jun0S2/AIBootcamp/blob/main/week4_openai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 필요한 라이브러리 설치
!pip install pandas numpy json

ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json


In [ ]:
 # 계속 1.xxx 버전에서 오류가 나서 다운그레이드 함
!pip install openai==0.28.0# 계속 1.xxx 버전에서 오류가 나서 다운그레이드 함
!pip show openai


Name: openai
Version: 0.28.0
Summary: Python client library for the OpenAI API
Home-page: https://github.com/openai/openai-python
Author: OpenAI
Author-email: support@openai.com
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, requests, tqdm
Required-by: 



api & json load

https://platform.openai.com/api-keys 에서 secret key 발급


In [ ]:
!pip install python-dotenv
from dotenv import load_dotenv
import os

# .env 파일 로드
load_dotenv()

In [ ]:
import json
import openai
import numpy as np
import asyncio

# GPT-4 API Key 설정
openai.api_key = os.getenv("OPENAI_API_KEY")

try:
    models = openai.Model.list()
    print("Available models:")
    for model in models["data"]:
        print(model["id"])
except Exception as e:
    print(f"Error retrieving models: {e}")


Available models:
dall-e-2
gpt-3.5-turbo
gpt-3.5-turbo-0125
gpt-3.5-turbo-instruct
babbage-002
whisper-1
dall-e-3
gpt-3.5-turbo-16k
omni-moderation-latest
o1-preview-2024-09-12
omni-moderation-2024-09-26
tts-1-hd-1106
o1-preview
tts-1-hd
davinci-002
text-embedding-ada-002
gpt-3.5-turbo-1106
tts-1
tts-1-1106
gpt-3.5-turbo-instruct-0914
text-embedding-3-small
gpt-4o-mini-2024-07-18
gpt-4o-mini
text-embedding-3-large
o1-mini
o1-mini-2024-09-12


In [ ]:
gpt 4o 는 리스트에 없어서 4o-mini 로 하였는데 뭔가 잘못한건가요..

In [ ]:
# api test

response = openai.ChatCompletion.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "나의 프롬프트 엔지니어링 코치가 되어줘."},
        {"role": "user", "content": "어떻게 하면 프롬프트 엔지니어링을 잘 할수 있을까?"}
    ]
)

print("Assistant: " + response.choices[0].message.content)


RateLimitError: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.

In [ ]:
# JSON 데이터 로드 함수
def load_kice_data(filepath):
    with open(filepath, "r", encoding="utf-8") as file:
        return json.load(file)

json -> GPT4 로 예측

In [ ]:
# 문제 예측 함수
def prediction(problem):
    """
    문제(json 형태)를 받아 GPT-4가 예측한 정답을 반환합니다.
    """
    # 문제 구성
    prompt = f"다음은 수능 국어 문제입니다:\n\n질문: {problem['question']}\n\n선택지:\n"
    for idx, choice in enumerate(problem['choices'], start=1):
        prompt += f"{idx}. {choice}\n"

    prompt += "\n정답을 선택지 번호로 답하세요."

    try:
        # GPT-4 API 호출
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "당신은 수능 국어 전문가입니다."},
                {"role": "user", "content": prompt}
            ]
        )

        # 응답에서 정답 추출
        gpt_response = response["choices"][0]["message"]["content"]
        answer = int(gpt_response.strip())
        return answer
    except ValueError:
        print(f"Invalid response: {gpt_response}")
        return None
    except Exception as e:
        print(f"API 호출 오류: {e}")
        return None

점수 계산

In [ ]:
def calculate_score(predictions, answers):
    """
    GPT-4 예측 결과와 실제 정답을 비교하여 점수를 계산합니다.
    """
    correct = sum([1 for pred, answer in zip(predictions, answers) if pred == answer])
    total = len(answers)
    return (correct / total) * 100

데이터 로드

In [ ]:
# 데이터 로드
print("Loading KICE data...")
# GitHub에서 데이터 다운로드
!git clone https://github.com/NomaDamas/KICE_slayer_AI_Korean.git

# JSON 데이터 로드
data = load_kice_data("KICE_slayer_AI_Korean/data/2023_11_KICE.json")
print("KICE data loaded successfully.")

# 데이터 타입 확인
print(type(data))  # 데이터 타입 확인 -> 리스트임
print(data[:5])    # 데이터의 첫 5개 항목 확인 (리스트인 경우)

# 문제와 정답 리스트 생성
problems = []
answers = []

# 데이터에서 문제와 정답을 추출
for item in data:
    for problem in item['problems']:
        problems.append(problem)
        answers.append(problem['answer'])  # 실제 정답 추가

print(f"Number of problems loaded: {len(problems)}")
# print(f"First problem: {problems[0]}")

Loading KICE data...
fatal: destination path 'KICE_slayer_AI_Korean' already exists and is not an empty directory.
KICE data loaded successfully.
<class 'list'>
[{'id': '2023_11_KICE_1-3', 'paragraph': '사람들이 지속적으로 책을 읽는 이유 중 하나는 즐거움이다. 독서의 즐거움에는 여러 가지가 있겠지만 그 중심에는 ‘소통의 즐거움’이 있다.독자는 독서를 통해 책과 소통하는 즐거움을 경험한다. 독서는필자와 간접적으로 대화하는 소통 행위이다. 독자는 자신이 속한사회나 시대의 영향 아래 필자가 속해 있거나 드러내고자 하는 사회나 시대를 경험한다. 직접 경험하지 못했던 다양한 삶을 필자를 매개로 만나고 이해하면서 독자는 더 넓은 시야로 세계를바라볼 수 있다. 이때 같은 책을 읽은 독자라도 독자의 배경지식이나 관점 등의 독자 요인, 읽기 환경이나 과제 등의 상황 요인이 다르므로, 필자가 보여 주는 세계를 그대로 수용하지 않고 저마다 소통 과정에서 다른 의미를 구성할 수 있다.[A] (이러한 소통은 독자가 책의 내용에 대해 질문하고 답을 찾아내는 과정에서 가능해진다. 독자는 책에서 답을 찾는 질문, 독자 자신에게서 답을 찾는 질문 등을 제기할 수 있다. 전자의 경우 책에 명시된 내용에서 답을 발견할 수 있고, 책의 내용들을 관계 지으며 답에 해당하는 내용을 스스로 구성할 수도 있다. 또한 후자의 경우 책에는 없는 독자의 경험에서 답을 찾을 수 있다. 이런 질문들을 풍부히 생성하고 주체적으로 답을 찾을 때 소통의 즐거움은 더 커진다.)한편 독자는 ㉠ (다른 독자와 소통하는 즐거움을 경험할 수도 있다.) 책과의 소통을 통해 개인적으로 형성한 의미를 독서 모임이나 독서 동아리 등에서 다른 독자들과 나누는 일이 이에 해당한다. 비슷한 해석에 서로 공감하며 기존 인식을 강화하거나 관점의 차이를 확인하고 기존 인식을 조정하는 

예측 테스트

In [ ]:
# GPT-4 예측 실행 테스트
test_problem = {
    "question": "다음 중 가장 큰 숫자는?",
    "choices": ["1", "2", "3", "4"]
}
print(f"Test Prediction: {prediction(test_problem)}")  # 4 출력 예상

# 전체 문제 예측 실행
print("Running predictions...")
predictions = []
for problem in problems:
    pred = prediction(problem)
    predictions.append(pred)

# 점수 계산
print("Calculating score...")
score = calculate_score(predictions, answers)

print(f"Final Score: {score}/100")
if score >= 80:
    print("✅ 80점 이상 조건 달성")
else:
    print("❌ 80점 미만")

API 호출 오류: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742

Test Prediction: None
Running predictions...
API 호출 오류: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


Rate limits


Copy page
Understand API rate limits and restrictions.
Rate limits are restrictions that our API imposes on the number of times a user or client can access our services within a specified period of time.

위 오류가 계속 나서 실행은 못했습니다.


In [ ]:
import json
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Hugging Face 모델 로드
model_name = "gpt2"  # GPT-2 모델 이름
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# JSON 데이터 로드 함수
def load_kice_data(filepath):
    """
    JSON 파일에서 데이터를 로드합니다.
    """
    with open(filepath, "r", encoding="utf-8") as file:
        return json.load(file)

# 문제 예측 함수
def prediction_with_huggingface(problem):
    """
    Hugging Face 모델을 사용하여 문제를 해결합니다.
    """
    # 문제 구성
    prompt = f"다음은 수능 국어 문제입니다:\n\n질문: {problem['question']}\n\n선택지:\n"
    for idx, choice in enumerate(problem['choices'], start=1):
        prompt += f"{idx}. {choice}\n"
    prompt += "\n정답은"

    # 모델의 최대 입력 길이를 초과하지 않도록 자르기
    max_input_length = model.config.n_positions - 50  # 최대 길이 - 예측에 필요한 공간 확보
    inputs = tokenizer.encode(prompt, return_tensors="pt", truncation=True, max_length=max_input_length)

    # 모델 예측
    outputs = model.generate(
        inputs,
        max_new_tokens=50,  # 모델이 생성할 최대 토큰 수
        num_return_sequences=1,
        do_sample=False,
        pad_token_id=tokenizer.eos_token_id  # 패딩 토큰 설정
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # 정답 번호 추출
    try:
        for idx, choice in enumerate(problem['choices'], start=1):
            if str(idx) in response:
                return idx
    except Exception as e:
        print(f"Error parsing response: {e}")
    return None

# 점수 계산 함수
def calculate_score(predictions, answers):
    """
    모델 예측 결과와 실제 정답을 비교하여 점수를 계산합니다.
    """
    correct = sum([1 for pred, answer in zip(predictions, answers) if pred == answer])
    total = len(answers)
    return (correct / total) * 100

# 데이터 로드
print("Loading KICE data...")
!git clone https://github.com/NomaDamas/KICE_slayer_AI_Korean.git || echo "Repository already cloned."

data = load_kice_data("KICE_slayer_AI_Korean/data/2023_11_KICE.json")
print("KICE data loaded successfully.")

# 문제와 정답 리스트 생성
problems = []
answers = []

for item in data:
    for problem in item["problems"]:
        problems.append(problem)
        answers.append(problem["answer"])

print(f"Number of problems loaded: {len(problems)}")

# GPT-2 예측 실행 테스트
print("Test Prediction...")
test_problem = {
    "question": "다음 중 가장 큰 숫자는?",
    "choices": ["1", "2", "3", "4"]
}
print(f"Test Prediction: {prediction_with_huggingface(test_problem)}")  # 4 출력 예상

# 전체 문제 예측 실행
print("Running predictions...")
predictions = []
for problem in problems:
    pred = prediction_with_huggingface(problem)
    predictions.append(pred)

# 점수 계산
print("Calculating score...")
score = calculate_score(predictions, answers)

print(f"Final Score: {score}/100")
if score >= 80:
    print("✅ 80점 이상 조건 달성")
else:
    print("❌ 80점 미만")


Loading KICE data...
fatal: destination path 'KICE_slayer_AI_Korean' already exists and is not an empty directory.
Repository already cloned.
KICE data loaded successfully.
Number of problems loaded: 45
Test Prediction...
Test Prediction: 1
Running predictions...
Calculating score...
Final Score: 17.77777777777778/100
❌ 80점 미만


open api 가 안써져서 gpt2 로 실행해 봤는데, 일단 정답률이 굉장히 낮습니다.